<a href="https://colab.research.google.com/github/eddieward21/transfer-learning/blob/main/InceptionV3_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [1]:
import os
import urllib.request
import random
import zipfile
from shutil import copyfile

#imports
from tensorflow.keras.optimizers import RMSprop #Root Mean Squared Propagation optimization algorithm
from tensorflow.keras.applications.inception_v3 import InceptionV3 #the model we will be "transferring learning" from
from tensorflow.keras import Model #starting point
from tensorflow.keras import layers #builds layers of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator #creates duplicates of our images that are flipped and rotated

